In [80]:
from PIL import Image
import numpy as np
import csv
import pandas
from matplotlib import pyplot as plt
import face_recognition
import keras
from keras.models import load_model
import cv2

NUM = 100

In [81]:
def Preprocess(filename):
    sheet = pandas.read_csv(filename)
    A = np.array(sheet.head(NUM))
    labels = []
    
    for i in range(NUM):
        #if not target emtions: 0, 3, 4
        if A[i][0] != 0 and A[i][0] != 3 and A[i][0] != 4:
            continue
        a = np.array(list(map(int,A[i][1].split(" ")))).reshape(48,48).astype(np.uint8)
        im = Image.fromarray(a)
        im.save(f"../test/{len(labels)}.jpeg")
        labels.append(A[i][0])
    return labels

In [82]:
def readFace(filename):
    face_image  = cv2.imread(filename)
    #plt.imshow(face_image)
    face_image = cv2.resize(face_image, (48,48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
    face_image = np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])
    return face_image

In [83]:
def Predict(length):
    trained_labels = []
    model = load_model("../emotion_detector_models/model_v6_23.hdf5")
    for i in range(length):
        face_image = readFace(f"../test/{i}.jpeg")
        predicted_class = np.argmax(model.predict(face_image))
        trained_labels.append(predicted_class)
    return trained_labels

In [92]:
def Accuracy(predict, answer):
    if len(predict) != len(answer):
        print("Not equal-length lists!")
        return
    angry_cnt , total_angry, happy_cnt , total_happy, neutral_cnt, total_neutral, total_cnt = 0, 0, 0, 0, 0, 0, 0
    angry = [0, 0, 0, 0, 0, 0, 0]
    happy = [0, 0, 0, 0, 0, 0, 0]
    neutral = [0, 0, 0, 0, 0, 0, 0]
    total = [0, 0, 0, 0, 0, 0, 0]
    Group = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
    
    for i in range(len(predict)):
        if predict[i] == answer[i]:
            isMatch = True
            total_cnt = total_cnt + 1
        else:
            isMatch = False
        
        total[predict[i]] = total[predict[i]] + 1
        if answer[i] == 0:
            angry[predict[i]] = angry[predict[i]] + 1
            total_angry = total_angry + 1
            if isMatch:
                angry_cnt = angry_cnt + 1
        elif answer[i] == 3:
            happy[predict[i]] = happy[predict[i]] + 1
            total_happy = total_happy + 1
            if isMatch:
                happy_cnt = happy_cnt + 1
        elif answer[i] == 4:
            neutral[predict[i]] = neutral[predict[i]] + 1
            total_neutral = total_neutral + 1
            if isMatch:
                neutral_cnt = neutral_cnt + 1
        if answer[i] == 3:
            if isMatch:
                Group[0][0] = Group[0][0] + 1
            elif predict[i] == 4 or predict[i] == 6:
                Group[0][1] = Group[0][1] + 1
            else:
                Group[0][2] = Group[0][2] + 1
        elif answer[i] == 4 or answer[i] == 6:
            if predict[i] == 3:
                Group[1][0] = Group[1][0] + 1
            elif predict[i] == 4 or predict[i] == 6:
                Group[1][1] = Group[1][1] + 1
            else:
                Group[1][2] = Group[1][2] + 1
        else:
            if predict[i] == 3:
                Group[2][0] = Group[2][0] + 1
            elif predict[i] == 4 or predict[i] == 6:
                Group[2][1] = Group[2][1] + 1
            else:
                Group[2][2] = Group[2][2] + 1
    
    for i in range(7):
        angry[i] = round(angry[i]/total_angry, 3)
        happy[i] = round(happy[i]/total_happy, 3)
        neutral[i] = round(neutral[i]/total_neutral, 3)
    for i in range(3):
        row_sum = 0
        for j in range(3):
            row_sum = row_sum + Group[i][j]
        for j in range(3):
            Group[i][j] = round(Group[i][j] / row_sum, 3)
            
    print(f"Emotion group distribution:")
    print(f"Positive: {Group[0]}")
    print(f"Neutral:  {Group[1]}")
    print(f"Negative: {Group[2]}")
    print(f"Target emotion distribution:")
    print(f"Angry:   {angry}")
    print(f"Happy:   {happy}")
    print(f"Neutral: {neutral}")
    print(f"Accuracy Rate:")
    print(f"Angry:   {angry_cnt} / {total_angry} = {round(angry_cnt/total_angry, 3)}")
    print(f"Happy:   {happy_cnt} / {total_happy} = {round(happy_cnt/total_happy, 3)}")
    print(f"Neutral: {neutral_cnt} / {total_neutral} = {round(neutral_cnt/total_neutral, 3)}")
    print(f"Total:   {total_cnt} / {len(answer)} = {round(total_cnt/len(answer), 3)}")

In [93]:
raw_data = "../train_data/train.csv"
answer = Preprocess(raw_data)
prediction = Predict(len(answer))
Accuracy(prediction, answer)

Emotion group distribution:
Positive: [0.3, 0.3, 0.4]
Neutral:  [0.0, 0.438, 0.562]
Negative: [0.056, 0.389, 0.556]
Target emotion distribution:
Angry:   [0.167, 0.0, 0.333, 0.056, 0.0, 0.056, 0.389]
Happy:   [0.0, 0.0, 0.4, 0.3, 0.0, 0.0, 0.3]
Neutral: [0.188, 0.0, 0.375, 0.0, 0.0, 0.0, 0.438]
Accuracy Rate:
Angry:   3 / 18 = 0.167
Happy:   6 / 20 = 0.3
Neutral: 0 / 16 = 0.0
Total:   9 / 54 = 0.167
